In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
if(torch.cuda.is_available()):
    device = torch.device('cuda')
elif(torch.backends.mps.is_available()):
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
IMAGE_SIZE = 32
batch_size = 32
mean, std = torch.tensor([0.4914, 0.4822, 0.4465]), torch.tensor([0.247, 0.243, 0.261])
# These values are mostly used by researchers as found to very useful in fast convergence


transform = transforms.Compose([
    transforms.RandomRotation(0.2) ,
    transforms.RandomHorizontalFlip() ,
    transforms.RandomVerticalFlip() ,
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])



train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size , shuffle = True)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size , shuffle = False)

classes = [ 'airplane', 'automobile', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']





In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
print(train_dataset.data.shape  , " - " , val_dataset.data.shape)
print(len(train_dataset.targets))

In [ ]:
# functions to show an image

def unnormalize(img):
        # unnormalize
    img[0]= img[0]*std[0] + mean[0]
    img[1]= img[1]*std[1] + mean[1]
    img[2]= img[2]*std[2] + mean[2]
    return img
    
def imshow(img):
    # unnormalize
    img = unnormalize(img)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def im_convert(tensor):
    img = tensor.cpu().clone().detach().numpy() #
    img = img.transpose(1, 2, 0)
    img = img * np.array(tuple(mean)) + np.array(tuple(std))
    img = img.clip(0, 1) # Clipping the size to print the images later
    return img


In [ ]:
fig = plt.figure(figsize=(20, 10)) 

for i in range(len(classes)):
    dataiter = iter(train_loader)
    images, labels = next(dataiter)
    for j in range (len(labels)):
        if(labels[j].item() == i):
            ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
            plt.imshow(im_convert(images[j]))
            ax.set_title(classes[i])



In [ ]:
for i in range(len(classes)):
    dataiter = iter(train_loader)
    images, labels = next(dataiter)
    for j in range (len(labels)):
        if(labels[j].item() == i):
            print(classes[i])
            imshow(images[j])
            break

In [ ]:
# class block(nn.Module):
#     def __init__(
#         self, in_channels, intermediate_channels, identity_downsample=None, stride=1
#     ):
#         super().__init__()
#         self.expansion = 4
#         self.conv1 = nn.Conv2d(
#             in_channels,
#             intermediate_channels,
#             kernel_size=1,
#             stride=1,
#             padding=0,
#             bias=False,
#         )
#         self.bn1 = nn.BatchNorm2d(intermediate_channels)
#         self.conv2 = nn.Conv2d(
#             intermediate_channels,
#             intermediate_channels,
#             kernel_size=3,
#             stride=stride,
#             padding=1,
#             bias=False,
#         )
#         self.bn2 = nn.BatchNorm2d(intermediate_channels)
#         self.conv3 = nn.Conv2d(
#             intermediate_channels,
#             intermediate_channels * self.expansion,
#             kernel_size=1,
#             stride=1,
#             padding=0,
#             bias=False,
#         )
#         self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
#         self.relu = nn.ReLU()
#         self.identity_downsample = identity_downsample
#         self.stride = stride

#     def forward(self, x):
#         identity = x.clone()

#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu(x)
#         x = self.conv3(x)
#         x = self.bn3(x)

#         if self.identity_downsample is not None:
#             identity = self.identity_downsample(identity)

#         x += identity
#         x = self.relu(x)
#         return x


# class ResNet(nn.Module):
#     def __init__(self, block, layers, image_channels, num_classes):
#         super(ResNet, self).__init__()
#         self.in_channels = 64
#         self.conv1 = nn.Conv2d(
#             image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
#         )
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU()
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

#         # Essentially the entire ResNet architecture are in these 4 lines below
#         self.layer1 = self._make_layer(
#             block, layers[0], intermediate_channels=64, stride=1
#         )
#         self.layer2 = self._make_layer(
#             block, layers[1], intermediate_channels=128, stride=2
#         )
#         self.layer3 = self._make_layer(
#             block, layers[2], intermediate_channels=256, stride=2
#         )
#         self.layer4 = self._make_layer(
#             block, layers[3], intermediate_channels=512, stride=2
#         )

#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(512 * 4, num_classes)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         x = self.avgpool(x)
#         x = x.reshape(x.shape[0], -1)
#         x = self.fc(x)

#         return x

#     def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
#         identity_downsample = None
#         layers = []

#         # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
#         # we need to adapt the Identity (skip connection) so it will be able to be added
#         # to the layer that's ahead
#         if stride != 1 or self.in_channels != intermediate_channels * 4:
#             identity_downsample = nn.Sequential(
#                 nn.Conv2d(
#                     self.in_channels,
#                     intermediate_channels * 4,
#                     kernel_size=1,
#                     stride=stride,
#                     bias=False,
#                 ),
#                 nn.BatchNorm2d(intermediate_channels * 4),
#             )

#         layers.append(
#             block(self.in_channels, intermediate_channels, identity_downsample, stride)
#         )

#         # The expansion size is always 4 for ResNet 50,101,152
#         self.in_channels = intermediate_channels * 4

#         # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
#         # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
#         # and also same amount of channels.
#         for i in range(num_residual_blocks - 1):
#             layers.append(block(self.in_channels, intermediate_channels))

#         return nn.Sequential(*layers)


# def ResNet50(img_channel=3, num_classes=1000):
#     return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


# def ResNet101(img_channel=3, num_classes=1000):
#     return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


# def ResNet152(img_channel=3, num_classes=1000):
#     return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super().__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels,
            intermediate_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            image_channels, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


def ResNet101(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


def ResNet152(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)

# Assuming 'device' is defined, as in your code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = ResNet101(3, 10)
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
model = ResNet101(3 , 10)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9)

In [ ]:
num_epochs = 5
# Training loop
train_acc = []
train_loss = []
val_acc = []
val_loss = []
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    model.train()  # Set the model to training mode

    for images, labels in train_loader:
        images = images.to(device)  # Move the input tensor to the GPU
        labels = labels.to(device)  # Move the labels tensor to the GPU

        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    training_loss = running_loss / len(train_loader)
    training_accuracy = 100 * correct_train / total_train
    

    # Evaluation on test set
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)  # Move the input tensor to the GPU
            labels = labels.to(device)  # Move the labels tensor to the GPU

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Calculate test accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_loss /= len(val_loader)
    test_accuracy = 100 * correct_test / total_test
    
    train_acc.append(training_accuracy)
    train_loss.append(training_loss)
    val_acc.append(test_accuracy)
    val_loss.append(test_loss)

    # Print the average loss and accuracy for this epoch
    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {training_loss:.4f} | Train Accuracy: {training_accuracy:.2f}%")
    print(f"  Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")
    print("*************************")

In [ ]:
final_state = model.state_dict()
torch.save(final_state, 'model_state.pth')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle(' ResNet Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,num_epochs+1))
ax1.plot(epoch_list, train_acc, label='Train Accuracy')
ax1.plot(epoch_list, val_acc, label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 60, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, train_loss, label='Train Loss')
ax2.plot(epoch_list, val_loss, label='Validation Loss')
ax2.set_xticks(np.arange(0, 60, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")